In [5]:
!pip install google-api-python-client google-auth google-auth-httplib2

In [15]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import base64
import os

try:
    
    print("Current Working Directory:", os.getcwd())
    print("Files in 'downloaded' directory:", os.listdir('./downloaded'))
    
    # Path to your service account key JSON file
    SERVICE_ACCOUNT_FILE = '../../gemini_service_account.json'
    
    # Authenticate with the service account
    credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE
    )
    
    # Initialize the Vision API client
    vision_service = build('vision', 'v1', credentials=credentials)
    
    # Path to the image file to analyze
    IMAGE_FILE = './downloaded/IMG_0253.JPG'
    if not os.path.exists(IMAGE_FILE):
        raise FileNotFoundError(f"File not found: {IMAGE_FILE}")
        
    # Encode the image to base64
    with open(IMAGE_FILE, 'rb') as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode('utf-8')
    
    # Prepare the request
    request_body = {
        'requests': [
            {
                'image': {'content': encoded_image},
                'features': [{'type': 'LABEL_DETECTION', 'maxResults': 10},
                             {'type': 'WEB_DETECTION', 'maxResults': 10}]
            }
        ]
    }
    
    # Send the request to the Vision API
    response = vision_service.images().annotate(body=request_body).execute()
    
    # Parse and display the response
    if 'responses' in response:
        for resp in response['responses']:
            print("Labels Detected:")
            for label in resp.get('labelAnnotations', []):
                print(f"  - {label['description']} (Score: {label['score']})")
            
            print("\nWeb Entities:")
            for web_entity in resp.get('webDetection', {}).get('webEntities', []):
                print(f"  - {web_entity['description']} (Score: {web_entity['score']})")
    else:
        print("No response received.")

    pass
except HttpError as error:
    print(f"An error occurred: {error}")


Current Working Directory: /Users/david/Documents/python/virtual/product_describer
Files in 'downloaded' directory: ['IMG_0253.JPG']
An error occurred: <HttpError 403 when requesting https://vision.googleapis.com/v1/images:annotate?alt=json returned "This API method requires billing to be enabled. Please enable billing on project #457521891066 by visiting https://console.developers.google.com/billing/enable?project=457521891066 then retry. If you enabled billing for this project recently, wait a few minutes for the action to propagate to our systems and retry.". Details: "[{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'BILLING_DISABLED', 'domain': 'googleapis.com', 'metadata': {'containerInfo': '457521891066', 'service': 'vision.googleapis.com', 'consumer': 'projects/457521891066'}}, {'@type': 'type.googleapis.com/google.rpc.LocalizedMessage', 'locale': 'en-US', 'message': 'This API method requires billing to be enabled. Please enable billing on project #457521891066 